<a href="https://colab.research.google.com/github/kimsh519/ms-ai-school/blob/main/0510_keras_04_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# 데이터 가져오기
from keras.datasets import boston_housing
(train_data,train_labels),(test_data,test_labels) = boston_housing.load_data()

In [40]:
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


In [41]:
print(train_labels[0])
print(train_data[0])

15.2
[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [42]:
# 데이터 전처리 (scaling)

mean = train_data.mean(axis = 0)
train_data -= mean # 전체 데이터에서 평균을 다 빼주었다
std = train_data.std(axis = 0)
train_data /= std
train_data[0]


array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [43]:
# 모델의 구성
from keras import models
from keras import layers

def build_model() :
  model = models.Sequential()
  model.add(layers.Dense(64, activation = 'relu', input_shape=(train_data.shape[1],)))
  model.add(layers.Dense(64, activation = 'relu'))
  model.add(layers.Dense(1))
  model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mse'])
  return model

In [44]:
# K-folder 검증을 사용한 훈련 검증 

import numpy as np 

k = 4                                       # folder 개수
num_val_samples = len(train_data) // k      # 한 folder의 데이터 수 
all_scores = []

for i in range(k):
  print('처리중인 폴드 #', i)

  # 검증 데이터의 준비 
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_labels = train_labels[i * num_val_samples: (i + 1) * num_val_samples]
   # i번째 폴드에 해당하는 데이터부터 i+1번쨰까지를 검증 데이터로 한다

  # 학습 데이터의 준비
  data1 = train_data[:i * num_val_samples]
  data2 = train_data[(i+1) * num_val_samples :]
  data1_labels = train_labels[:i * num_val_samples]
  data2_labels = train_labels[(i+1) * num_val_samples :]
  # 학습 데이터와 검증 데이터를 분리하기 위해서 concatenate 함수를 사용해 병합한다.
  partial_train_data = np.concatenate([data1, data2], axis=0)
  partial_train_labels = np.concatenate([data1_labels, data2_labels], axis=0)
  # axis = 0 을 추가함으로서 행을 기준으로 병합시켰다

  # 모델의 학습
  model = build_model()
  model.summary()
  history = model.fit(partial_train_data, partial_train_labels, epochs=500, batch_size=128, verbose=0)
  #verbose = 0  : 각각의 epochs 결과를 skip하고 결론만 보여준다

  # 모델의 검증
  val_mse, val_mae = model.evaluate(val_data, val_labels)
  print(val_mse, val_mae)
  all_scores.append(val_mae)
  


처리중인 폴드 # 0
Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 64)                896       
                                                                 
 dense_63 (Dense)            (None, 64)                4160      
                                                                 
 dense_64 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 0s 3ms/step - loss: 8.1940 - mse: 8.1940
8.193965911865234 8.193965911865234
처리중인 폴드 # 1
Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)   

In [45]:
np.mean(all_scores)

11.236572980880737